<a href="https://colab.research.google.com/github/elhassn/AI_QA_PDF_Colab_Notebook/blob/main/AI_QA_PDF_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-Powered Q&A over PDF Files
This notebook demonstrates a prototype that:
- Parses uploaded PDFs
- Embeds the content using OpenAI
- Stores it in Pinecone
- Answers questions via LangChain

In [ ]:
# Install dependencies
!pip install langchain openai pypdf pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 8.1 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Set API keys
import os
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
os.environ['PINECONE_API_KEY'] = 'PINECONE_API_KEY'
os.environ['PINECONE_ENV'] = 'us-east'

In [ ]:
# Upload and read PDF
from google.colab import files
from PyPDF2 import PdfReader

uploaded = files.upload()
pdf_path = next(iter(uploaded))

reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text()
print(text[:1000])

Saving Mid-term+Project_Mohamed+El+hassnaoui.pdf to Mid-term+Project_Mohamed+El+hassnaoui.pdf
Mid-term
 
Project
 
-
 
Product
 
Review
 
Sentiment
 
Analysis
 
 
 
 
Given
 
model
 
instructions
 
and
 
context
 
-
  
Tested
 
on
 
Chat
 
playground
 
 
 
 
 
 
 
 
 
 
 
 
 
You
 
are
 
an
 
expert
 
Product
 
Analysis
 
Assistant
 
for
 
a
 
multinational
 
computer,
 
phone,
 
laptop,
 
and
 
hardware
 
manufacturer.
 
Your
 
primary
 
responsibility
 
is
 
to
 
analyze
 
customer
 
reviews
 
of
 
the
 
company’s
 
products
 
and
 
services,
 
extract
 
valuable
 
insights,
 
and
 
structure
 
this
 
information
 
to
 
help
 
the
 
product
 
management
 
team
 
prioritize
 
improvements.
 
You
 
have
 
deep
 
expertise
 
in
 
understanding
 
customer
 
sentiment,
 
identifying
 
product
 
issues,
 
and
 
recognizing
 
competitive
 
advantages
 
in
 
the
 
technology
 
hardware
 
market.
 
 
 
 
 
 
 
Goal
 
and
 
Objective
 
Your
 
goal
 
is
 
to
 
analyze
 
unstructured
 
customer


In [ ]:
!pip uninstall -y pinecone-client  # Uninstall the old version
!pip install pinecone-client  # Install the new version (renamed 'pinecone')


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
  Using cached pinecone_client-6.0.0-py3-none-any.whl.metadata (3.4 kB)
Using cached pinecone_client-6.0.0-py3-none-any.whl (6.7 kB)


In [ ]:
!pip uninstall -y pinecone-client  # Uninstall the old version of Pinecone
!pip install pinecone  # Install the updated version (just `pinecone`)


Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 10.8 MB/s eta 0:00:00


In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone instance
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

# Now, you can list indexes and create one if needed
index_name = "pdf-qa-index"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # or your vector dimension
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',  # Change to your preferred cloud
            region='us-east-1'  # Change to your desired region
        )
    )


In [ ]:
from pinecone import Pinecone

# Create a Pinecone instance with your API key
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])

# Use the instance to interact with Pinecone
# Example: list existing indexes
print(pc.list_indexes().names())


['pdf-qa-index']


In [ ]:
os.environ['PINECONE_API_KEY'] = 'your-pinecone-api-key'
os.environ['PINECONE_ENV'] = 'us-east-1'  # e.g., 'us-west1-gcp'


In [ ]:
# Correct index name with lowercase and hyphens only
index_name = 'pdf-qa-index'

# Check if the index exists, and create it if necessary
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Or your vector dimension
        metric='euclidean',
        spec=ServerlessSpec(cloud='aws', region='us-west-2')
    )